<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<font size='1'>Modificado el 2017</font>
</p>

El polimorfismo "se refiere a la propiedad por la que es posible enviar mensajes sintácticamente iguales a objetos de tipos distintos" ([Wikipedia](https://es.wikipedia.org/wiki/Polimorfismo_(inform%C3%A1tica), 2017). Básicamente es utilizar distintos objetos de tipos distintos con la misma *interfaz*. Overriding y Overloading son dos maneras de hacer polimorfismo.

- Overriding: ocurre cuando se implementa un método en una subclase que "invalida" la implementación del mismo método en la super clase
   
- Overloading: es la capacidad de definir un método con el mismo nombre pero con distinto número y tipo de argumentos. Es la capacidad de una función de ejecutar distintas acciones dependiendo del tipo y número de argumentos que recibe. 
  
Python no soporta *overloading* (en realidad no es necesario), se puede "simular" usando algunos parámetros con valores por defecto o número de argumentos variables, pero no se puede definir la función más de una vez con distintos tipos y números de argumentos y esperar que ambas definiciones sean consideradas por el programa.
   

#### Ejemplo

La clase $\texttt{Variable}$ representa un conjunto de datos cualquiera, mientras que la subclase $\texttt{Ingresos}$ contiene un método para calcular el valor "representante" (algo así como el promedio, mediana, moda, etc.). Ocurre lo mismo con las subclases $\texttt{Comuna}$ y $\texttt{Puesto}$: 

Si los datos corresponden a ingresos, el representante es el promedio. 
Si los datos corresponden a la comuna, el representante es la comuna que más se repite. 
Finalmente, si los datos corresponden al puesto de trabajo, entonces el representante es el que tiene el puesto más alto según la jerarquía especificada en el diccionario "categorías".

In [1]:
import numpy  as np

class Variable:
    
    def __init__(self, data):
        self.data = np.array(data)

    def representante(self):
        pass


class Ingresos(Variable):
    
    def representante(self):
        return np.mean(self.data)


class Comuna(Variable):
    
    def representante(self):
        ind = np.argmax([np.sum(self.data == c) for c in self.data])  # el que mas se repite
        return self.data[ind]


class Puesto(Variable):
    
    categorias = {'Gerente': 1, 'SubGerente': 2, 'Analista': 3, 
                  'Alumno en Practica': 4} # class (or static) variable

    def representante(self):
        return self.data[np.argmin([Puesto.categorias[c] for c in self.data])]#la categoria mas alta acorde con el diccionario



In [3]:
lista_pesos = Ingresos([50, 80, 90, 150, 45, 65, 78, 89, 59, 77, 90])
lista_comunas = Comuna(['Providencia', 'Macul' , 'LaReina' ,'Santiago', 'Providencia', 'PuenteAlto',
                        'Macul', 'Santiago', 'Santiago' ])
lista_puestos = Puesto(['SubGerente', 'Analista','SubGerente','Analista','Alumno en Practica',
                        'Alumno en Practica'])


print(lista_pesos.representante())
print(lista_comunas.representante())
print(lista_puestos.representante())


79.3636363636
Santiago
SubGerente


<h1> Overriding de operadores en Python </h1>

Existen muchos operadores en Python que funcionan para varias de las clases "built-in". Por ejemplo, el operador "+" puede sumar dos números, concatenar dos strings, mezclar dos listas, etc. dependiendo de la clase con la que estemos trabajando:

In [20]:
a = [1,2,3,4]
b = [5,6,7,8]
print(a+b)
c = "Hola"
d = " Mundo"
print(c+d)

[1, 2, 3, 4, 5, 6, 7, 8]
Hola Mundo


Nosotros también podemos personalizar el método `__add__` para que funcione en algún tipo de clase específica que necesitemos. Por ejemplo, supongamos una clase que representa un carro de compra:

In [4]:
class Carro:
    '''
    Un carro de compras lo representaremos como un diccionario 
    donde el key es el nombre del producto y el value es la cantidad
    Ej: {'pan' : 3, 'leche' : 2, 'agua' : 6}
    '''
    
    def __init__(self, lista_productos):
        self.lista_productos = lista_productos

    def __add__(self, otro_carro):
        lista_sumada = self.lista_productos
        for p in otro_carro.lista_productos.keys():  # aquí vamos recorriendo los nombres de los productos
            if p in self.lista_productos.keys():
                lista_sumada.update({ p : otro_carro.lista_productos[p] + self.lista_productos[p]})  # aquí creo la nueva instancia con las cantidades sumada
            else:
                lista_sumada.update({ p : otro_carro.lista_productos[p]})
                
        return Carro(lista_sumada)
    
    def __repr__(self):
        return "\n".join("Producto: {} | Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())
    
    

In [22]:
carro_1 = Carro({'pan' : 3, 'leche' : 2, 'agua' : 6})
carro_2 = Carro({'leche' : 5, 'bebida' : 2, 'cerveza' : 12})
carro_3 = carro_1 + carro_2
print(carro_3.lista_productos)

{'bebida': 2, 'agua': 6, 'cerveza': 12, 'pan': 3, 'leche': 7}


 El método `__repr__` nos permite generar un string que será usado a la hora de llamar a print de alguna instancia de Carro:

In [23]:
print(carro_3)

Producto: bebida | Cantidad: 2
Producto: agua | Cantidad: 6
Producto: cerveza | Cantidad: 12
Producto: pan | Cantidad: 3
Producto: leche | Cantidad: 7


También podríamos haber implementado el método \_\_str\_\_ que cumple la misma función que el método \_\_repr\_\_. La principal diferencia es que \_\_repr\_\_ debería contener todos los detalles necesarios para identificar bien al objeto, como para ser usado por alguien que implementará algo en el futuro y debe entender bien nuestro código. El método \_\_str\_\_ está orientado a generar una impresión "human-readable", algo que se vea bien y se interprete bien en el contexto en particular, pero no necesariamente debe contener todos los detalles técnicos del objeto. En casos en que \_\_str\_\_ está implementado, print usará el string generado por \_\_str\_\_ para imprimir, pero cuando no esté \_\_str\_\_ implementado, print usará el método \_\_repr\_\_.

In [1]:
class Carro:
    ''' Un carro de compras lo representaremos como un diccionario 
        donde el key es el nombre del producto y el value es la cantidad
        Ej: {'pan' : 3, 'leche' : 2, 'agua' : 6}
    '''
    def __init__(self, lista_productos):
        self.lista_productos = lista_productos

    def __add__(self, otro_carro):
        lista_sumada = self.lista_productos
        for p in otro_carro.lista_productos.keys():#aquí vamos recorriendo los nombres de los productos
            if p in self.lista_productos.keys():
                lista_sumada.update({ p : otro_carro.lista_productos[p] + self.lista_productos[p]})#aquí creo la nueva instancia con las cantidades sumada
            else:
                lista_sumada.update({ p : otro_carro.lista_productos[p]})
                
        return Carro(lista_sumada)
    
    def __repr__(self):
        s = self.__doc__#esto retorna el string del comienzo de la clase, la documentación que la describe
        return s + "\n" + "\n".join("Producto: {} | Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())
    
    def __str__(self):
        return "\n".join("Producto: {} - Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())    

In [2]:
carro_1 = Carro({'pan' : 3, 'leche' : 2, 'agua' : 6})
carro_2 = Carro({'leche' : 5, 'bebida' : 2, 'cerveza' : 12})
carro_3 = carro_1 + carro_2
print(carro_3)

Producto: pan - Cantidad: 3
Producto: bebida - Cantidad: 2
Producto: leche - Cantidad: 7
Producto: agua - Cantidad: 6
Producto: cerveza - Cantidad: 12


Si comentamos el método \_\_str\_\_, print va a imprimir el string que retorna la función \_\_repr\_\_

In [3]:
class Carro:
    ''' Un carro de compras lo representaremos como un diccionario 
        donde el key es el nombre del producto y el value es la cantidad
        Ej: {'pan' : 3, 'leche' : 2, 'agua' : 6}
    '''
    def __init__(self, lista_productos):
        self.lista_productos = lista_productos

    def __add__(self, otro_carro):
        lista_sumada = self.lista_productos
        for p in otro_carro.lista_productos.keys():#aquí vamos recorriendo los nombres de los productos
            if p in self.lista_productos.keys():
                lista_sumada.update({ p : otro_carro.lista_productos[p] + self.lista_productos[p]})#aquí creo la nueva instancia con las cantidades sumada
            else:
                lista_sumada.update({ p : otro_carro.lista_productos[p]})
                
        return Carro(lista_sumada)
    
    def __repr__(self):
        s = self.__doc__#esto retorna el string del comienzo de la clase, la documentación que la describe
        return s + "\n" + "\n".join("Producto: {} | Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())
    
#    def __str__(self):
#        return "\n".join("Producto: {} - Cantidad: {}".format(p, self.lista_productos[p]) for p in self.lista_productos.keys())    

In [4]:
carro_1 = Carro({'pan' : 3, 'leche' : 2, 'agua' : 6})
carro_2 = Carro({'leche' : 5, 'bebida' : 2, 'cerveza' : 12})
carro_3 = carro_1 + carro_2
print(carro_3)

 Un carro de compras lo representaremos como un diccionario 
        donde el key es el nombre del producto y el value es la cantidad
        Ej: {'pan' : 3, 'leche' : 2, 'agua' : 6}
    
Producto: pan | Cantidad: 3
Producto: bebida | Cantidad: 2
Producto: leche | Cantidad: 7
Producto: agua | Cantidad: 6
Producto: cerveza | Cantidad: 12


De la misma forma podemos personalizar la mayoría de los operadores, por ejemplo, para personalizar el operador "menor que" (less than):

In [5]:
class Punto: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y
    
    def __lt__(self, otro_punto): 
        self_mag = (self.x ** 2) + (self.y ** 2) 
        otro_punto_mag = (otro_punto.x ** 2) + (otro_punto.y ** 2) 
        return self_mag < otro_punto_mag

p1 = Punto(2,4)
p2 = Punto(8,3)
print(p1 < p2)

True


# Duck Typing

> if it walks like a duck and quacks like a duck then it is a duck" 
(no importa el tipo de objeto si contiene la acción)
 
Duck typing es una característica de algunos lenguajes que hace que el polimorfismo sea menos atractivo, ya que el lenguaje por sí sólo es capaz de generar comportamiento polimórfico sin la necesidad de implementar el polimorfismo a través de la herencia. 

In [2]:
class Pato:
    
    def gritar(self):
        print("Quack!")
        
    def caminar(self):
        print("caminando como un pato...")        
    
class Persona:
    
    def gritar(self):
        print("Ahhh!")
        
    def caminar(self):
        print("caminando como un humano...")        

        
def activar(pato): #esto en otro tipo de lenguaje obligaría a que pato sea del tipo "Pato", por lo tanto
    pato.gritar() #la función activar no podría ser llamada con un argumento tipo "Persona"
    pato.caminar()


donald = Pato()
juan = Persona()
activar(donald)
activar(juan)

Quack!
caminando como un pato...
Ahhh!
caminando como un humano...
